In [1]:
# evaluate predictions
verbose=True

EVAL = False 
PLOT_LOSS = False

num_workers = 0

import config
print('results in:', config.EVAL_IMG_PATHS)

results in: /Users/jnaiman/Downloads/tmp/wormfindr/wormOntologyNSF/multiClassWorm256/eval_images/


In [2]:
import torch
from model import UNET
#from utils import *
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision import transforms
#from cityscapesscripts.helpers.labels import trainId2label as t2l
from torch.utils.data import DataLoader


from dataset import BossDBSliceDataset
# for bossdb
from intern import array

# for updates
from importlib import reload
import config
reload(config)
from numpy import random, column_stack

# get dataset URI
boss_uri_images = config.IMAGE_DATASET_URI
boss_uri_masks = config.SEGMASK_DATASET_URI
# JPN -- have to get dataset size and centroids
channel = array(boss_uri_images)
# get shape
shape = channel.shape
print('dataset shape=', shape)

# select a set of centroids randomly
ncentroid_train = config.NCENTROID_TRAIN
ncentroid_test = config.NCENTROID_TEST
zshift = config.ZSHIFT

if torch.cuda.is_available():
    device = 'cuda:0'
    print('Running on the GPU')
else:
    device = 'cpu'
    print('Running on the CPU')

/opt/anaconda3/envs/WormOntology/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dataset shape= (303, 8346, 12830)
Running on the CPU


In [3]:
def save_as_images(tensor_pred, folder, image_name, multiclass=True):
    if y.shape[0] == 1: # only one layer
        tensor_pred = transforms.ToPILImage()(tensor_pred.byte())
        filename = f"{folder}\{image_name}.png"
        tensor_pred.save(filename)
    else:
        for i in range(tensor_pred.shape[0]): # for each one
            tensor_pred_out = transforms.ToPILImage()(tensor_pred[i,:,:].byte())
            image_name_out = image_name + '_batch' + str(i)
            filename = f"{folder}\{image_name_out}.png"
            filename = filename.replace(r'//',r'/')
            filename = filename.replace('\\','')
            tensor_pred_out.save(filename)

In [4]:
def save_predictions(data, model, location):    
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(data)):

            #X, y, s = batch # here 's' is the name of the file stored in the root directory
            X, y = batch # JPN
            X, y = X.to(device), y.to(device)
            # save images
            save_as_images(X, location, 'image_' + str(idx), multiclass=True)   
            # save gt             
            save_as_images(y, location, 'gtmask_' + str(idx), multiclass=True)   

            predictions = model(X) 
            
            predictions = torch.nn.functional.softmax(predictions, dim=1)
            pred_labels = torch.argmax(predictions, dim=1) 
            pred_labels = pred_labels.float()

            # Remapping the labels
            pred_labels = pred_labels.to('cpu')
            ##pred_labels.apply_(lambda x: t2l[x].id)
            pred_labels = pred_labels.to(device)   

            # Resizing predicted images too original size
            #pred_labels = transforms.Resize((1024, 2048))(pred_labels)             

            # Configure filename & location to save predictions as images
            save_as_images(pred_labels, location, 'predMask_' + str(idx), multiclass=True)                


In [5]:
# get test paths
z = random.randint(config.ZMIN,config.ZMAX-1,size=ncentroid_test)
y = random.randint(config.YMIN+config.INPUT_IMAGE_WIDTH//2,config.YMAX-config.INPUT_IMAGE_WIDTH//2,size=ncentroid_test)
x = random.randint(config.XMIN+config.INPUT_IMAGE_WIDTH//2,config.XMAX-config.INPUT_IMAGE_WIDTH//2,size=ncentroid_test)
centroids_test = column_stack([z,y,x])

# save test centroids
path_save = config.EVAL_IMG_PATHS
print("[INFO] saving test centroids to "+path_save+" ...")
f = open(path_save, "w")
#print(centroids_test.astype('str').tolist())
for t in centroids_test:
    f.write(str(t)+'\n')
#f.write("\n".join(centroids_test.astype('str').tolist()))
f.close()

# create the train and test datasets
testDS = BossDBSliceDataset(image_boss_uri=boss_uri_images, mask_boss_uri=boss_uri_masks, 
                                centroid_list_zyx = centroids_test, 
                                boss_config = None, # None for public dataset
                                px_radius_yx = [config.INPUT_IMAGE_WIDTH//2,config.INPUT_IMAGE_WIDTH//2],
                                zshift=zshift, verbose=verbose)
val_set = DataLoader(testDS, shuffle=False,
    batch_size=config.BATCH_SIZE, pin_memory=config.PIN_MEMORY,
    num_workers=num_workers)

[INFO] saving test centroids to /Users/jnaiman/Downloads/tmp/wormfindr/wormOntologyNSF/multiClassWorm256/test_paths.txt ...
image,mask uri= bossdb://white1986/n2u/em bossdb://white1986/n2u/seg


In [ ]:
print('Data has been loaded!')

net = UNET(in_channels=3, classes=config.NUM_CLASSES).to(device)
checkpoint = torch.load(config.MODEL_PATH)
net.load_state_dict(checkpoint['model_state_dict'])
net.eval()
print(f'{config.MODEL_PATH} has been loaded and initialized')
save_predictions(val_set, net, config.EVAL_IMG_PATHS)

Data has been loaded!


/var/folders/t7/bwcvd_4177q4872gxghn7p9r0000gq/T/ipykernel_30146/1356453059.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(config.MODEL_PATH)


/Users/jnaiman/Downloads/tmp/wormfindr/wormOntologyNSF/multiClassWorm256/unet_model.pth has been loaded and initialized


 50%|█████     | 1/2 [04:37<04:37, 277.79s/it]

In [7]:
# for idx, batch in enumerate(tqdm(val_set)):
#     import sys; sys.exit()

In [8]:
#X,y = batch

In [44]:
#y.shape[0]

In [45]:
#np.unique(y)